In [2]:
# Load libraries

import os
import pandas 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pySuStaIn
import statsmodels.formula.api as smf
from scipy import stats
import sklearn.model_selection
from sklearn.linear_model import LinearRegression

/home/robin/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## All biomarker inputs 

In [10]:
# Combining all biomarkers into one csv

# Merge data frames
finalEBM_input = pandas.merge(cog_data, hippo_vol_data, left_index=True, right_index=True)
finalEBM_input = pandas.merge(finalEBM_input, meta_tau_data, left_index=True, right_index=True)

#finalEBM_input = finalEBM_input[['RID','Diagnosis_x', 'W-ADAS11', 'W-average_hippo', 'W-Average_meta_tau']]
finalEBM_input = finalEBM_input[['RID','Diagnosis_x', 'ADAS11_bl', 'Average_hippo', 'Average_meta_tau']]

finalEBM_input = pandas.merge(finalEBM_input, tau_data, left_index=True, right_index=True)
finalEBM_input = pandas.merge(finalEBM_input, vol_data, left_index=True, right_index=True)
finalEBM_input = finalEBM_input.drop(['RID_x','Diagnosis_x','RID_y'], axis=1)
#finalEBM_input = pandas.merge(finalEBM_input, lobe_vol_data, left_index=True, right_index=True)

# Reorder columns
#finalEBM_input = finalEBM_input[['RID', 'Diagnosis', 'AGE', 'W-ADAS11', 'W-average_hippo', 'W-Average_meta_tau', 'W-ctx_entorhinal_tau','W-ctx_parahippocampal_tau', 'W-Hippocampus_tau', 'W-Amygdala_tau', 'W-ctx_precuneus_tau' , 'W-ctx_anteriorcingulate_tau' , 'W-Average_superiorfrontal_vol' , 'W-Average_anteriorcingulate_vol' ,'W-Average_posteriorcingulate_vol' ,'W-Average_precuneus_vol' ,'W-Average_inferiorparietal_vol' ,'W-Average_inferiortemporal_vol']]
finalEBM_input = finalEBM_input[['RID', 'Diagnosis', 'AGE', 'ADAS11_bl', 'Average_hippo', 'Average_meta_tau', 'ctx_entorhinal_tau','ctx_inferiorparietal_tau','ctx_inferiortemporal_tau','ctx_parahippocampal_tau', 'Hippocampus_tau', 'Amygdala_tau', 'ctx_precuneus_tau' , 'ctx_anteriorcingulate_tau' , 'Average_superiorfrontal_vol' , 'Average_anteriorcingulate_vol' ,'Average_posteriorcingulate_vol' ,'Average_precuneus_vol' ,'Average_inferiorparietal_vol' ,'Average_inferiortemporal_vol']]

# Delete redundant RIDs
finalEBM_input = finalEBM_input.drop_duplicates(subset='RID', keep='first')

finalEBM_input


,RID,Diagnosis,AGE,ADAS11_bl,Average_hippo,Average_meta_tau,ctx_entorhinal_tau,ctx_inferiorparietal_tau,ctx_inferiortemporal_tau,ctx_parahippocampal_tau,Hippocampus_tau,Amygdala_tau,ctx_precuneus_tau,ctx_anteriorcingulate_tau,Average_superiorfrontal_vol,Average_anteriorcingulate_vol,Average_posteriorcingulate_vol,Average_precuneus_vol,Average_inferiorparietal_vol,Average_inferiortemporal_vol
0,21,CN,72.6,-0.172429,0.062109,0.134515,-0.011721,-0.175590,0.356465,-0.215946,2.382817,-0.093166,-0.597984,0.121476,0.449522,0.301755,0.532955,0.757263,-0.239702,1.704690
1,31,CN,77.7,-1.735559,2.046008,-0.392692,0.623041,-0.477772,-0.668344,-0.214256,0.428236,0.057401,-0.282911,-0.197098,0.819599,0.170225,0.503228,-0.052603,0.279301,1.286340
3,56,MCI,69.6,-1.319766,0.710701,0.456155,1.153732,0.418942,0.147350,0.695881,0.934010,0.272791,0.946754,0.601243,0.502718,2.723248,0.207082,-0.088145,0.967623,1.773868
5,59,MCI,70.9,-0.069262,0.581602,0.544075,0.635106,0.399789,0.297354,0.546652,2.263903,0.557123,1.181402,0.491065,1.084545,0.817633,0.672696,0.396530,1.746611,1.869215
6,69,MCI,72.9,0.346530,0.370436,1.093217,0.857338,0.732524,0.930174,1.370171,0.191038,1.150125,1.114470,0.874382,-0.376692,-0.057383,0.429549,-0.127077,0.361946,0.004851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812,6704,MCI,65.3,3.160164,0.470892,-0.188450,-0.323373,0.191931,-0.064490,-0.385318,-0.551500,-0.765101,0.653389,0.361317,1.752011,1.306636,0.455020,0.965912,1.024877,1.925646
813,6705,MCI,74.9,0.971782,2.098817,1.965910,2.678315,1.139518,1.376567,1.959302,1.049724,2.596439,1.583092,0.237739,-0.001617,1.134429,0.922593,-0.098058,0.080927,-0.253239
814,6713,CN,71.1,0.971782,0.521152,-0.249232,-0.260623,0.234837,-0.127469,-0.497668,-0.369991,-0.220408,0.645313,0.412271,-0.221338,0.301398,-1.040388,-0.699135,-0.754194,-0.464371
815,6721,Dementia,77.9,3.785416,2.874163,-0.762779,-0.695716,-1.116853,-0.748597,-0.866402,-1.031006,-0.272775,-1.108548,-0.852030,0.185680,1.569582,-0.350836,0.463439,-0.699300,-0.347036


In [11]:
# Save finalEBM_input to a csv
finalEBM_input.to_csv('finalEBM_input_no_ageregression.csv')

## Regional Tau Input
Entorhinal, parahippocampal, hippocampal, amygdala, precuneus, anterior cingulate

In [3]:
tau_data = pandas.read_csv('Cog_Tau_V1.csv')

# Change every '-' to '_' in column title names (to avoid errors later on)
tau_data.columns = [c.replace('-', '_') for c in tau_data.columns]

# Find the average of the right and left cerebellum 
tau_data['Average_Cerebellum'] = tau_data[['Left_Cerebellum_Cortex','Right_Cerebellum_Cortex']].mean(axis=1)

# Divide each regional tau value by the average of R and L cerebellum tau and subtract 1
corrected = (tau_data.iloc[:, 7:].div(tau_data['Average_Cerebellum'], axis=0) - 1).values
tau_data.iloc[:, 7:] = corrected

# Calculate average for each region between hemispheres
tau_data['ctx_entorhinal_tau'] = tau_data[['ctx_rh_entorhinal', 'ctx_lh_entorhinal']].mean(axis=1)
tau_data['ctx_parahippocampal_tau'] = tau_data[['ctx_rh_parahippocampal', 'ctx_lh_parahippocampal']].mean(axis=1)
tau_data['Hippocampus_tau'] = tau_data[['Right_Hippocampus', 'Left_Hippocampus']].mean(axis=1)
tau_data['Amygdala_tau'] = tau_data[['Right_Amygdala', 'Left_Amygdala']].mean(axis=1)
tau_data['ctx_precuneus_tau'] = tau_data[['ctx_rh_precuneus', 'ctx_lh_precuneus']].mean(axis=1)
tau_data['ctx_anteriorcingulate_tau'] = tau_data[['ctx_rh_rostralanteriorcingulate', 'ctx_rh_caudalanteriorcingulate', 'ctx_lh_rostralanteriorcingulate', 'ctx_lh_caudalanteriorcingulate']].mean(axis=1)
tau_data['ctx_inferiortemporal_tau'] = tau_data[['ctx_rh_inferiortemporal', 'ctx_lh_inferiortemporal']].mean(axis=1)
tau_data['ctx_inferiorparietal_tau'] = tau_data[['ctx_rh_inferiorparietal', 'ctx_lh_inferiorparietal']].mean(axis=1)

# Add in columns with demographic data so we can perform age regression 
with_demo = pandas.read_csv('Regional_Tau_w_Demographics.csv')
with_demo = with_demo[['AGE', 'RID', 'merge_DX']]

# Housekeeping
tau_data = tau_data[['merge_RID', 'ctx_entorhinal_tau', 'ctx_inferiorparietal_tau','ctx_inferiortemporal_tau','ctx_parahippocampal_tau', 'Hippocampus_tau', 'Amygdala_tau', 'ctx_precuneus_tau', 'ctx_anteriorcingulate_tau']]
tau_data = tau_data.rename(columns={'merge_RID': 'RID'})
with_demo = with_demo.rename(columns={'AGE':'Age'})

# Merge with demographic data
tau_data = pandas.merge(tau_data, with_demo, left_index=True, right_index=True)
tau_data = tau_data.rename(columns={'RID_x': 'RID'})
tau_data = tau_data.rename(columns={'merge_DX': 'Diagnosis'})
tau_data = tau_data.drop(['RID_y'],axis=1)

# Drop rows where all value is NaN 
value_to_drop = [4513, 6038, 6505, 6598]  
tau_data = tau_data[~tau_data['RID'].isin(value_to_drop)]

# # Age regression
# model = LinearRegression()

# # Container for residuals
# residuals_dict = {}

# for col in tau_data.columns[1:7]:  
#     # Fit model
#     X = tau_data[['Age']]
#     y = tau_data[col]
#     model.fit(X, y)
    
#     # Predict
#     y_pred = model.predict(X)
    
#     # Calculate residuals and store them
#     residuals_dict[col] = y - y_pred

# residuals_df = pandas.DataFrame(residuals_dict)

# for col in residuals_df.columns:
#     tau_data[f'W-{col}'] = residuals_df[col]
    
#tau_data = tau_data[['RID','Diagnosis', 'W-ctx_entorhinal_tau', 'W-ctx_parahippocampal_tau', 'W-Hippocampus_tau', 'W-Amygdala_tau', 'W-ctx_precuneus_tau', 'W-ctx_anteriorcingulate_tau']]
tau_data = tau_data[['RID','Diagnosis', 'ctx_entorhinal_tau', 'ctx_inferiorparietal_tau','ctx_inferiortemporal_tau','ctx_parahippocampal_tau', 'Hippocampus_tau', 'Amygdala_tau', 'ctx_precuneus_tau', 'ctx_anteriorcingulate_tau']]


# extract data for control subjects
tau_data_control = tau_data[tau_data['Diagnosis'] == 'CN']

for col in tau_data.columns[2:]:  
    
    # compute the mean and standard deviation of the control population
    mean_control = tau_data_control[[col]].mean()
    std_control = tau_data_control[[col]].std()

    # z-score the data
    tau_data[[col]] = (tau_data[[col]]-mean_control)/std_control
    
tau_data

,RID,Diagnosis,ctx_entorhinal_tau,ctx_inferiorparietal_tau,ctx_inferiortemporal_tau,ctx_parahippocampal_tau,Hippocampus_tau,Amygdala_tau,ctx_precuneus_tau,ctx_anteriorcingulate_tau
0,21,CN,-0.011721,-0.175590,0.356465,-0.215946,2.382817,-0.093166,-0.597984,0.121476
1,31,CN,0.623041,-0.477772,-0.668344,-0.214256,0.428236,0.057401,-0.282911,-0.197098
2,31,CN,0.613513,-0.592824,-0.747840,0.000465,0.783426,0.430698,-0.220524,-0.506216
3,56,MCI,1.153732,0.418942,0.147350,0.695881,0.934010,0.272791,0.946754,0.601243
4,56,MCI,0.972820,0.910451,0.285803,0.875521,1.429403,0.601653,1.566434,1.014943
...,...,...,...,...,...,...,...,...,...,...
814,6704,CN,-0.260623,0.234837,-0.127469,-0.497668,-0.369991,-0.220408,0.645313,0.412271
815,6705,Dementia,-0.695716,-1.116853,-0.748597,-0.866402,-1.031006,-0.272775,-1.108548,-0.852030
816,6713,Dementia,2.305189,-0.871174,0.101877,1.252488,0.883321,2.791833,-1.515187,-1.511726
817,6721,Dementia,0.600853,0.513453,0.531595,0.932595,0.648436,0.679912,1.190048,1.054012


## Regional Volumes

In [4]:
# All volumes already z-scored, negative

vol_data = pandas.read_csv('residual_neg_z_scores.csv')

# Drop rows where all value is NaN 
value_to_drop = [4513, 6038, 6505, 6598]  
vol_data = vol_data[~vol_data['RID'].isin(value_to_drop)]

# Take averages of left and right superior frontal gyrus
vol_data['Average_superiorfrontal_vol'] = vol_data[['ctx-lh-superiorfrontal','ctx-rh-superiorfrontal']].mean(axis=1)

# Sum anterior cingulate subregions and average left and right hemis
vol_data['ctx-lh-anteriorcingulate'] = vol_data[['ctx-lh-rostralanteriorcingulate', 'ctx-lh-caudalanteriorcingulate']].sum(axis=1)
vol_data['ctx-rh-anteriorcingulate'] = vol_data[['ctx-rh-rostralanteriorcingulate','ctx-rh-caudalanteriorcingulate']].sum(axis=1)
vol_data['Average_anteriorcingulate_vol'] = vol_data[['ctx-lh-anteriorcingulate','ctx-rh-anteriorcingulate']].mean(axis=1)

# Take averages of left and right posterior cingulate cortices
vol_data['Average_posteriorcingulate_vol'] = vol_data[['ctx-lh-posteriorcingulate','ctx-rh-posteriorcingulate']].mean(axis=1)

# Take average of left and right precuneous
vol_data['Average_precuneus_vol'] = vol_data[['ctx-rh-precuneus','ctx-lh-precuneus']].mean(axis=1)

# Take average of left and right inferior parietal lobules
vol_data['Average_inferiorparietal_vol'] = vol_data[['ctx-rh-inferiorparietal','ctx-lh-inferiorparietal']].mean(axis=1)

# Take average of left and right inferior temporal gyrus
vol_data['Average_inferiortemporal_vol'] = vol_data[['ctx-rh-inferiortemporal','ctx-lh-inferiortemporal']].mean(axis=1)

# Delete unneeded rows
vol_data = vol_data[['RID','AGE','Average_superiorfrontal_vol','Average_anteriorcingulate_vol','Average_posteriorcingulate_vol','Average_precuneus_vol','Average_inferiorparietal_vol','Average_inferiortemporal_vol']]

# # Age regression
# model = LinearRegression()

# # Container for residuals
# residuals_dict = {}

# for col in vol_data.columns[2:]:  
#     # Fit model
#     X = vol_data[['AGE']]
#     y = vol_data[col]
#     model.fit(X, y)
    
#     # Predict
#     y_pred = model.predict(X)
    
#     # Calculate residuals and store them
#     residuals_dict[col] = y - y_pred

# residuals_df = pandas.DataFrame(residuals_dict)

# for col in residuals_df.columns:
#     vol_data[f'W-{col}'] = residuals_df[col]

#vol_data = vol_data[['RID','AGE','W-Average_superiorfrontal_vol','W-Average_anteriorcingulate_vol','W-Average_posteriorcingulate_vol','W-Average_precuneus_vol','W-Average_inferiorparietal_vol','W-Average_inferiortemporal_vol']]
vol_data = vol_data[['RID','AGE','Average_superiorfrontal_vol','Average_anteriorcingulate_vol','Average_posteriorcingulate_vol','Average_precuneus_vol','Average_inferiorparietal_vol','Average_inferiortemporal_vol']]
          
vol_data


,RID,AGE,Average_superiorfrontal_vol,Average_anteriorcingulate_vol,Average_posteriorcingulate_vol,Average_precuneus_vol,Average_inferiorparietal_vol,Average_inferiortemporal_vol
0,21,72.6,0.449522,0.301755,0.532955,0.757263,-0.239702,1.704690
1,31,77.7,0.819599,0.170225,0.503228,-0.052603,0.279301,1.286340
2,31,77.7,0.819599,0.170225,0.503228,-0.052603,0.279301,1.286340
3,56,69.6,0.502718,2.723248,0.207082,-0.088145,0.967623,1.773868
4,56,69.6,0.502718,2.723248,0.207082,-0.088145,0.967623,1.773868
...,...,...,...,...,...,...,...,...
812,6704,65.3,1.752011,1.306636,0.455020,0.965912,1.024877,1.925646
813,6705,74.9,-0.001617,1.134429,0.922593,-0.098058,0.080927,-0.253239
814,6713,71.1,-0.221338,0.301398,-1.040388,-0.699135,-0.754194,-0.464371
815,6721,77.9,0.185680,1.569582,-0.350836,0.463439,-0.699300,-0.347036


## Hippocampal Volume Input

In [5]:
# Preparing hippocampal volume data (already z-scored, neg, and age regressed)

hippo_vol_data = pandas.read_csv('residual_neg_z_scores.csv')

# Delete unneeded rows 
#hippo_vol_data = hippo_vol_data[['RID', 'W-RHV', 'W-LHV']]
hippo_vol_data = hippo_vol_data[['RID', 'Right-Hippocampus', 'Left-Hippocampus']]

# Take average of right and left hippocmapal hemisphere columes 
#hippo_vol_data['W-average_hippo'] = hippo_vol_data[['W-RHV', 'W-LHV']].mean(axis=1)
hippo_vol_data['Average_hippo'] = hippo_vol_data[['Right-Hippocampus', 'Left-Hippocampus']].mean(axis=1)

# Delete unneeded rows
#hippo_vol_data = hippo_vol_data[['RID','W-average_hippo']]
hippo_vol_data = hippo_vol_data[['RID','Average_hippo']]

# Drop rows where all value is NaN 
value_to_drop = [4513, 6038, 6505, 6598]    
hippo_vol_data = hippo_vol_data[~hippo_vol_data['RID'].isin(value_to_drop)]

hippo_vol_data

,RID,Average_hippo
0,21,0.062109
1,31,2.046008
2,31,2.046008
3,56,0.710701
4,56,0.710701
...,...,...
812,6704,0.470892
813,6705,2.098817
814,6713,0.521152
815,6721,2.874163


## Lobe Volumes Input

In [7]:
# All volumes already z-scored, negative

lobe_vol_data = pandas.read_csv('residual_neg_z_scores.csv')

# Take average of all regions in the frontal lobe (including left and right)
lobe_vol_data['Average_frontal'] = lobe_vol_data[['ctx-rh-caudalmiddlefrontal', 'ctx-rh-lateralorbitofrontal', 'ctx-rh-medialorbitofrontal', 'ctx-rh-rostralmiddlefrontal', 'ctx-rh-superiorfrontal', 'ctx-rh-frontalpole', 'ctx-lh-caudalmiddlefrontal','ctx-lh-lateralorbitofrontal','ctx-lh-medialorbitofrontal','ctx-lh-rostralmiddlefrontal', 'ctx-lh-superiorfrontal', 'ctx-lh-frontalpole']].mean(axis=1)

# Take average of all regions in the temporal lobe (including left and right)
lobe_vol_data['Average_temporal'] = lobe_vol_data[['ctx-rh-inferiortemporal', 'ctx-rh-middletemporal','ctx-rh-superiortemporal','ctx-rh-temporalpole','ctx-rh-transversetemporal','ctx-lh-inferiortemporal','ctx-lh-middletemporal','ctx-lh-superiortemporal','ctx-lh-temporalpole','ctx-lh-transversetemporal']].mean(axis=1)

# Take average of all regions in the parietal lobe (including left and right)
lobe_vol_data['Average_parietal'] = lobe_vol_data[['ctx-rh-inferiorparietal','ctx-rh-superiorparietal','ctx-lh-inferiorparietal','ctx-lh-superiorparietal']].mean(axis=1)

# Take average of all regions in the occipital lobe (including left and right)
lobe_vol_data['Average_occipital'] = lobe_vol_data[['ctx-rh-lateraloccipital','ctx-lh-lateraloccipital']].mean(axis=1)

# Delete unneeded rows
lobe_vol_data = lobe_vol_data[['RID','AGE','Average_frontal','Average_temporal','Average_parietal','Average_occipital']]

# Drop rows where all value is NaN 
value_to_drop = [4513, 6038, 6505, 6598]  
lobe_vol_data = lobe_vol_data[~lobe_vol_data['RID'].isin(value_to_drop)]

# # Age regression
# model = LinearRegression()

# # Container for residuals
# residuals_dict = {}

# for col in lobe_vol_data.columns[2:]:  
#     # Fit model
#     X = lobe_vol_data[['AGE']]
#     y = lobe_vol_data[col]
#     model.fit(X, y)
    
#     # Predict
#     y_pred = model.predict(X)
    
#     # Calculate residuals and store them
#     residuals_dict[col] = y - y_pred

# residuals_df = pandas.DataFrame(residuals_dict)

# for col in residuals_df.columns:
#     lobe_vol_data[f'W-{col}'] = residuals_df[col]

#lobe_vol_data = lobe_vol_data[['RID', 'W-Average_frontal', 'W-Average_temporal', 'W-Average_parietal', 'W-Average_occipital']]
lobe_vol_data = lobe_vol_data[['RID', 'Average_frontal', 'Average_temporal', 'Average_parietal', 'Average_occipital']]

lobe_vol_data


,RID,Average_frontal,Average_temporal,Average_parietal,Average_occipital
0,21,0.339853,1.036789,0.601625,1.100572
1,31,0.414346,0.930375,0.573440,0.406073
2,31,0.414346,0.930375,0.573440,0.406073
3,56,0.676851,0.821165,0.721422,1.261718
4,56,0.676851,0.821165,0.721422,1.261718
...,...,...,...,...,...
812,6704,1.535122,1.152178,0.608225,1.467432
813,6705,0.368088,0.194553,0.006670,-0.165769
814,6713,-0.077413,-0.294780,-0.816025,-0.392363
815,6721,0.156117,-0.058634,0.073827,0.219533


## Meta ROI Tau Input

In [6]:
# Preparing tau data

meta_tau_data = pandas.read_csv('Cog_Tau_V1.csv')

# Find the average of the right and left cerebellum 
meta_tau_data['Average_Cerebellum'] = (meta_tau_data['Left-Cerebellum-Cortex']+ meta_tau_data['Right-Cerebellum-Cortex'])/2

# Change every '-' to '_' in column title names (to avoid errors later on)
meta_tau_data.columns = [c.replace('-', '_') for c in meta_tau_data.columns]

# Divide each regional tau value by the average of R and L cerebellum tau and subtract 1
meta_tau_data[['ctx_rh_inferiortemporal', 'ctx_rh_middletemporal', 'ctx_rh_parahippocampal', 'ctx_lh_entorhinal', 'ctx_lh_fusiform', 'ctx_lh_inferiortemporal', 'ctx_lh_middletemporal', 'ctx_lh_parahippocampal', 'Left_Amygdala', 'Right_Amygdala']] = meta_tau_data[['ctx_rh_inferiortemporal', 'ctx_rh_middletemporal', 'ctx_rh_parahippocampal', 'ctx_lh_entorhinal', 'ctx_lh_fusiform', 'ctx_lh_inferiortemporal', 'ctx_lh_middletemporal', 'ctx_lh_parahippocampal', 'Left_Amygdala', 'Right_Amygdala']].div(meta_tau_data['Average_Cerebellum'], axis=0)
meta_tau_data[['ctx_rh_inferiortemporal', 'ctx_rh_middletemporal', 'ctx_rh_parahippocampal', 'ctx_lh_entorhinal', 'ctx_lh_fusiform', 'ctx_lh_inferiortemporal', 'ctx_lh_middletemporal', 'ctx_lh_parahippocampal', 'Left_Amygdala', 'Right_Amygdala']] = meta_tau_data[['ctx_rh_inferiortemporal', 'ctx_rh_middletemporal', 'ctx_rh_parahippocampal', 'ctx_lh_entorhinal', 'ctx_lh_fusiform', 'ctx_lh_inferiortemporal', 'ctx_lh_middletemporal', 'ctx_lh_parahippocampal', 'Left_Amygdala', 'Right_Amygdala']] - 1

# Take average of all meta tau regions
meta_tau_data['Average_meta_tau'] = meta_tau_data[['ctx_rh_inferiortemporal', 'ctx_rh_middletemporal', 'ctx_rh_parahippocampal', 'ctx_lh_entorhinal', 'ctx_lh_fusiform', 'ctx_lh_inferiortemporal', 'ctx_lh_middletemporal', 'ctx_lh_parahippocampal', 'Left_Amygdala', 'Right_Amygdala']].mean(axis=1)

# Add in columns with demographic data so we can perform age regression 
with_demo = pandas.read_csv('Regional_Tau_w_Demographics.csv')
with_demo = with_demo[['AGE', 'RID']]

# Rename column titles for consistency 
meta_tau_data = meta_tau_data.rename(columns={'merge_RID': 'RID'})
with_demo = with_demo.rename(columns={'AGE':'Age'})

meta_tau_data = pandas.merge(meta_tau_data, with_demo, left_index=True, right_index=True)
meta_tau_data = meta_tau_data[['RID_x', 'merge_DX', 'Age','Average_meta_tau']]
meta_tau_data = meta_tau_data.rename(columns={'RID_x': 'RID'})
meta_tau_data = meta_tau_data.rename(columns={'merge_DX': 'Diagnosis'})

# Drop rows where all value is NaN 
value_to_drop = [4513, 6038, 6505, 6598]  
meta_tau_data = meta_tau_data[~meta_tau_data['RID'].isin(value_to_drop)]

meta_tau_data = meta_tau_data[['RID','Diagnosis', 'Age', 'Average_meta_tau']]

# # Extracting the columns for regression analysis
# X = meta_tau_data[['Age']]  # Independent variable
# y = meta_tau_data['Average_meta_tau']  # Dependent variable

# # Fitting the simple linear regression model
# model = LinearRegression()
# model.fit(X, y)

# # Predicting the values using the fitted model
# y_pred = model.predict(X)

# # Calculating residuals
# residuals = y - y_pred

# # Adding the residuals as a new column named 'W-Average_Tau' to the dataframe
# meta_tau_data['W-Average_meta_tau'] = residuals

# Inclue only the relevant columns  
#meta_tau_data = meta_tau_data[['RID', 'Diagnosis', 'W-Average_meta_tau']]
meta_tau_data = meta_tau_data[['RID', 'Diagnosis', 'Average_meta_tau']]

# extract data for control subjects
meta_tau_data_control = meta_tau_data[meta_tau_data['Diagnosis'] == 'CN']
    
# compute the mean and standard deviation of the control population
# meta_tau_mean_control = meta_tau_data_control[['W-Average_meta_tau']].mean()
# meta_tau_std_control = meta_tau_data_control[['W-Average_meta_tau']].std()
meta_tau_mean_control = meta_tau_data_control[['Average_meta_tau']].mean()
meta_tau_std_control = meta_tau_data_control[['Average_meta_tau']].std()

# z-score the data
#meta_tau_data[['W-Average_meta_tau']] = (meta_tau_data[['W-Average_meta_tau']]-meta_tau_mean_control)/meta_tau_std_control
meta_tau_data[['Average_meta_tau']] = (meta_tau_data[['Average_meta_tau']]-meta_tau_mean_control)/meta_tau_std_control


meta_tau_data

,RID,Diagnosis,Average_meta_tau
0,21,CN,0.134515
1,31,CN,-0.392692
2,31,CN,-0.311040
3,56,MCI,0.456155
4,56,MCI,0.612054
...,...,...,...
814,6704,CN,-0.249232
815,6705,Dementia,-0.762779
816,6713,Dementia,1.093864
817,6721,Dementia,0.610988


## Cognitive Score Input

In [7]:
# Preparing cognitive score input
cog_data = pandas.read_csv('Regional_Tau_w_Demographics.csv')


### Using ADAS11

In [8]:
# Rename columns for consistency
cog_data = cog_data.rename(columns={'merge_DX': 'Diagnosis'})
cog_data = cog_data.rename(columns={'AGE': 'Age'})

# # Drop rows where all value is NaN 
value_to_drop = [4513, 6038, 6505, 6598]  
cog_data = cog_data[~cog_data['RID'].isin(value_to_drop)]

# Delete unneeded columns 
cog_data = cog_data[['RID', 'Age', 'Diagnosis', 'ADAS11_bl']]

# extract data for control subjects
cog_data_control = cog_data[cog_data['Diagnosis'] == 'CN']

# compute the mean and standard deviation of the control population
cog_mean_control = cog_data_control[['ADAS11_bl']].mean()
cog_std_control = cog_data_control[['ADAS11_bl']].std()

# z-score the data
cog_data[['ADAS11_bl']] = (cog_data[['ADAS11_bl']]-cog_mean_control)/cog_std_control

# # Age regression for cognitive score

# # Extracting the columns for regression analysis
# X = cog_data[['Age']]  # Independent variable
# y = cog_data['ADAS11_bl']  # Dependent variable

# # Fitting the simple linear regression model
# model = LinearRegression()
# model.fit(X, y)

# # Predicting the 'MMSE_bl' values using the fitted model
# y_pred = model.predict(X)

# # Calculating residuals
# residuals = y - y_pred

# # Adding the residuals as a new column 
# cog_data['W-ADAS11'] = residuals

# Delete unneeded columns
#cog_data = cog_data[['RID', 'Diagnosis', 'W-ADAS11']]
cog_data = cog_data[['RID', 'Diagnosis', 'ADAS11_bl']]

cog_data

,RID,Diagnosis,ADAS11_bl
0,21,CN,-0.172429
1,31,CN,-1.735559
2,31,CN,-1.735559
3,56,MCI,-1.319766
4,56,MCI,-1.319766
...,...,...,...
814,6704,CN,0.971782
815,6705,Dementia,3.785416
816,6713,Dementia,3.682250
817,6721,Dementia,3.891709


### Using MMSE

In [3]:
# Rename columns for consistency
data = data.rename(columns={'merge_DX': 'Diagnosis'})
data = data.rename(columns={'AGE': 'Age'})

# Change diagnosis to numerical values
mapping = {'CN': 0, 'MCI': 1, 'Dementia': 2}
data['Diagnosis'] = data['Diagnosis'].map(mapping)

# Delete unneeded columns 
data = data[['RID', 'Age', 'Diagnosis', 'MMSE_bl']]

# extract data for control subjects
data_control = data[data['Diagnosis'] == 0]

# compute the mean and standard deviation of the control population
mean_control = data_control[['MMSE_bl']].mean()
std_control = data_control[['MMSE_bl']].std()

# z-score the data
data[['MMSE_bl']] = (data[['MMSE_bl']]-mean_control)/std_control

# Compute 30 - z-score
data[['MMSE_bl']] = 30 -data[['MMSE_bl']]

In [3]:
# Drop rows where all value is NaN 
value_to_drop = [4513, 6038, 6505, 6598]  
    
data = data[~data['RID'].isin(value_to_drop)]

      RID  Age  Diagnosis    MMSE_bl
734  6505  NaN          0  30.111267
778  6598  NaN          1  33.827959


In [20]:
# Age regression for cognitive score

from sklearn.linear_model import LinearRegression

# Extracting the columns for regression analysis
X = data[['Age']]  # Independent variable
y = data['MMSE_bl']  # Dependent variable

# Fitting the simple linear regression model
model = LinearRegression()
model.fit(X, y)

# Predicting the 'MMSE_bl' values using the fitted model
y_pred = model.predict(X)

# Calculating residuals
residuals = y - y_pred

# Adding the residuals as a new column named 'W-MMSE' to the dataframe
data['W-MMSE'] = residuals

# Delete unneeded columns
data = data[['RID', 'Diagnosis', 'W-MMSE']]
data

/tmp/ipykernel_394364/3052876416.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['W-MMSE'] = residuals


,RID,Diagnosis,W-MMSE
0,21,0,-1.448714
1,31,0,-1.577825
2,31,0,-1.577825
3,56,1,0.485579
4,56,1,0.485579
...,...,...,...
814,6704,0,0.594438
815,6705,2,3.138925
816,6713,2,1.376779
817,6721,2,3.062977


In [21]:
data.to_csv('Cogscore.csv')